In [1]:
"""
This notebook is used to perform RFECV and assemble the resulting feature lists (and those from feature_selection_test.ipynb) into a dictionary to be pickled and used later.

It also provides basic hyperparameter search spaces for our classifiers. Spaces here will be updated as we do more work and find better tuning ranges.

Author: Wesley
"""
import pandas as pd
import numpy as np
from skopt.space import Categorical, Integer, Real

from sklearn.feature_selection import RFECV

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.svm import LinearSVC

import pickle

from sklearn.multiclass import OneVsRestClassifier

import traceback

In [3]:
multiclass_train = pd.read_csv("multiclass_train.csv")
multiclass_test = pd.read_csv("multiclass_test.csv")
binary_train = pd.read_csv("binary_train.csv")
binary_test = pd.read_csv("binary_test.csv")
binary = pd.concat([binary_train, binary_test])
multiclass = pd.concat([multiclass_train, multiclass_test])

We perform RFE using RFECV for all classifiers for which this is possible. Binary is first.

In [4]:
# Encode binary labels and prepare X and y
binary[" Label"] = binary[" Label"].apply(lambda x: 0 if x == "BENIGN" else 1)
y = binary[" Label"].copy()
X = binary.drop([" Label"], axis=1)

le = LabelEncoder()
multiclass[" Label"] = le.fit_transform(multiclass[" Label"].values)

multiclass_labels = []
print("\nMulticlass Label Encodings (in order of digits 0 -> n): ")
for i in range(0, len(list(set(list(multiclass[' Label']))))):
    multiclass_labels.append(le.inverse_transform([i])[0])

print(multiclass_labels)

multiclass_y = multiclass[" Label"].copy()
multiclass_X = multiclass.drop([" Label"], axis=1)


Multiclass Label Encodings (in order of digits 0 -> n): 
['DNS', 'LDAP', 'MSSQL', 'NTP', 'NetBIOS', 'Portmap', 'SNMP', 'SSDP', 'Syn', 'TFTP', 'UDP', 'UDP-lag']


In [9]:
models = {
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42),
    "Logistic Regression": make_pipeline(StandardScaler(), LogisticRegression(random_state=42)),
}

# This is a suitable scoring method because the classes are balanced.
score_method = 'accuracy'

print("Binary")
for name, model in models.items():
    if name == "Linear SVC":
        rfecv = RFECV(
            estimator=model,
            scoring=score_method,
            n_jobs=5,
            importance_getter="named_steps.linearsvc.coef_",
        )

    elif name == "Logistic Regression":
        rfecv = RFECV(
            estimator=model,
            scoring=score_method,
            n_jobs=5,
            importance_getter="named_steps.logisticregression.coef_",
        )

    else:            
        rfecv = RFECV(
            estimator=model,
            scoring=score_method,
            n_jobs=5,
        )

    try:      
        rfecv.fit(X, y)
        features = X.columns[rfecv.support_].tolist()
        print(f"{name}: {features}")

    except:
        print(f"{name} failed to generate features.\n")

print("Multiclass")
for name, model in models.items():
    if name == "Linear SVC":
        rfecv = RFECV(
            estimator=model,
            scoring=score_method,
            n_jobs=5,
            importance_getter="named_steps.linearsvc.coef_",
        )

    elif name == "Logistic Regression":
        rfecv = RFECV(
            estimator=model,
            scoring=score_method,
            n_jobs=5,
            importance_getter="named_steps.logisticregression.coef_",
        )

    else:            
        rfecv = RFECV(
            estimator=model,
            scoring=score_method,
            n_jobs=5,
        )

    try:      
        rfecv.fit(multiclass_X, multiclass_y)
        features = multiclass_X.columns[rfecv.support_].tolist()
        print(f"{name}: {features}")

    except:
        traceback.print_stack()
        print(f"{name} failed to generate features.\n")

Binary
Decision Tree: [' Protocol', ' Flow Duration', ' Total Fwd Packets', ' Total Backward Packets', 'Total Length of Fwd Packets', ' Total Length of Bwd Packets', ' Fwd Packet Length Max', ' Fwd Packet Length Min', ' Fwd Packet Length Mean', ' Fwd Packet Length Std', 'Bwd Packet Length Max', ' Bwd Packet Length Min', ' Bwd Packet Length Mean', ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max', ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std', ' Fwd Header Length', ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s', ' Min Packet Length', ' Max Packet Length', ' Packet Length Variance', 'FIN Flag Count', ' SYN Flag Count', ' URG Flag Count', ' CWE Flag Count', ' ECE Flag Count', ' Down/Up Ratio', ' Average Packet Size', ' Avg Fwd Segment Size', ' Fwd Header Length.1', 'Bwd Avg Bulk Rate', ' Subflow Fwd Bytes', ' Subflow B

c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/

Logistic Regression: [' Total Fwd Packets', ' Total Length of Bwd Packets', ' Fwd Packet Length Max', ' Fwd Packet Length Min', ' Fwd Packet Length Mean', 'Bwd Packet Length Max', ' Bwd Packet Length Min', ' Bwd Packet Length Mean', ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Min', ' Fwd IAT Mean', ' Fwd IAT Min', ' Bwd IAT Std', ' Bwd IAT Max', 'Fwd PSH Flags', ' Fwd Header Length', ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean', ' Packet Length Std', ' Packet Length Variance', ' ACK Flag Count', ' URG Flag Count', ' CWE Flag Count', ' Average Packet Size', ' Avg Fwd Segment Size', ' Avg Bwd Segment Size', ' Fwd Header Length.1', 'Subflow Fwd Packets', ' Subflow Bwd Packets', ' Subflow Bwd Bytes', 'Init_Win_bytes_forward', ' Init_Win_bytes_backward', 'Active Mean', ' Active Std', ' Active Max', ' Active Min', 'Idle Mean', ' Idle Min', ' Inbound']
Multiclass
Decision Tree: [' Flow Duration', ' Total Fwd Packets', ' Total Ba

c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/

Logistic Regression: [' Protocol', ' Flow Duration', 'Total Length of Fwd Packets', ' Fwd Packet Length Min', ' Flow IAT Mean', ' Flow IAT Std', ' Fwd IAT Std', ' Fwd IAT Max', ' Min Packet Length', ' Packet Length Mean', ' Packet Length Std', ' Packet Length Variance', ' Average Packet Size', ' Subflow Fwd Bytes', ' Subflow Bwd Packets', ' act_data_pkt_fwd', 'Idle Mean', ' Idle Max', ' Idle Min']


c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Multiclass is next. LinearSVC was abandoned due to being too slow

Assembled search spaces and feature lists.

In [5]:
# Hyperparameter search spaces for eligible classifiers. Currently set to generic ranges, will be updates as we test them.
search_spaces = {   
    "Logistic Regression": {       
            "logisticregression__C": Real(0.00001, 10000, prior="log-uniform"),
            "logisticregression__max_iter": Integer(100, 500),
    },
    "Random Forest": {
            "criterion": Categorical(["gini", "entropy"]),
            "max_depth": Integer(3, 100),
            "max_features": Categorical(["sqrt", "log2"]),
            "min_samples_leaf": Integer(1, 20),
            "min_samples_split": Integer(2, 20),
            "n_estimators": Integer(50, 500),
    },
    "KNN": {
            "n_neighbors": Integer(2,50),
            "weights": Categorical(["uniform","distance"]),
            "algorithm": Categorical(["ball_tree","kd_tree", "brute"]),
    },
    "Naive Bayes": {
        "var_smoothing": Real(1e-9,1e-7, prior="log-uniform")
    },
    "Decision Tree": {
            "criterion": Categorical(["gini", "entropy"]),
            "max_depth": Integer(3, 50),
            "max_features": Categorical(["sqrt", "log2"]),
            "min_samples_leaf": Integer(1, 10),
            "min_samples_split": Integer(2, 10),
    },
}

# Contains all currently chosen feature sets from RFE, MI, and Pearson Correlation-Based Feature Selection. Also contains chosen variance threshold for PCA.
feature_lists = {
    "Binary": {
        "All": X.columns,
        "Correlation" : [' Protocol', ' Fwd Packet Length Max', ' Fwd Packet Length Min', ' Fwd Packet Length Mean', 'Bwd Packet Length Max', ' Bwd Packet Length Min', ' Bwd Packet Length Mean', ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s', 'Fwd PSH Flags', 'Fwd Packets/s', ' Min Packet Length', ' Packet Length Mean', ' Packet Length Std', ' Packet Length Variance', ' RST Flag Count', ' URG Flag Count', ' CWE Flag Count', ' Down/Up Ratio', ' Average Packet Size', ' Avg Fwd Segment Size', ' Avg Bwd Segment Size', 'Init_Win_bytes_forward', ' Init_Win_bytes_backward', ' Inbound'],
        "Mutual Information": [' Protocol', ' Flow Duration', ' Total Fwd Packets', ' Total Backward Packets', 'Total Length of Fwd Packets', ' Total Length of Bwd Packets', ' Fwd Packet Length Max', ' Fwd Packet Length Min', ' Fwd Packet Length Mean', ' Fwd Packet Length Std', 'Bwd Packet Length Max', ' Bwd Packet Length Min', ' Bwd Packet Length Mean', 'Flow Bytes/s', ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max', ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Max', ' Bwd IAT Min', ' Fwd Header Length', ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s', ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean', ' Packet Length Std', ' Packet Length Variance', ' URG Flag Count', ' Down/Up Ratio', ' Average Packet Size', ' Avg Fwd Segment Size', ' Avg Bwd Segment Size', ' Fwd Header Length.1', 'Subflow Fwd Packets', ' Subflow Fwd Bytes', ' Subflow Bwd Packets', ' Subflow Bwd Bytes', 'Init_Win_bytes_forward', ' Init_Win_bytes_backward', ' act_data_pkt_fwd', ' min_seg_size_forward', ' Inbound'],
        "RFE Sets": {
                "Decision Tree": [' Protocol', ' Flow Duration', ' Total Fwd Packets', ' Total Backward Packets', 'Total Length of Fwd Packets', ' Total Length of Bwd Packets', ' Fwd Packet Length Max', ' Fwd Packet Length Min', ' Fwd Packet Length Mean', ' Fwd Packet Length Std', 'Bwd Packet Length Max', ' Bwd Packet Length Min', ' Bwd Packet Length Mean', ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max', ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std', ' Fwd Header Length', ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s', ' Min Packet Length', ' Max Packet Length', ' Packet Length Variance', 'FIN Flag Count', ' SYN Flag Count', ' URG Flag Count', ' CWE Flag Count', ' ECE Flag Count', ' Down/Up Ratio', ' Average Packet Size', ' Avg Fwd Segment Size', ' Fwd Header Length.1', 'Bwd Avg Bulk Rate', ' Subflow Fwd Bytes', ' Subflow Bwd Packets', ' Subflow Bwd Bytes', 'Init_Win_bytes_forward', ' Init_Win_bytes_backward', ' act_data_pkt_fwd', ' min_seg_size_forward', 'Active Mean', ' Active Std', ' Active Min', 'Idle Mean', ' Idle Max', ' Idle Min', 'SimillarHTTP', ' Inbound'],
                "Random Forest": [' Total Backward Packets', 'Total Length of Fwd Packets', ' Fwd Packet Length Min', ' Fwd Packet Length Mean', ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Max', ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s', ' Min Packet Length', ' Packet Length Mean', ' Packet Length Variance', ' ACK Flag Count', ' URG Flag Count', ' Average Packet Size', ' Avg Fwd Segment Size', ' Subflow Bwd Packets', 'Init_Win_bytes_forward', ' Idle Max', ' Inbound'],
                "Logistic Regression": [' Total Fwd Packets', ' Total Length of Bwd Packets', ' Fwd Packet Length Max', ' Fwd Packet Length Min', ' Fwd Packet Length Mean', 'Bwd Packet Length Max', ' Bwd Packet Length Min', ' Bwd Packet Length Mean', ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Min', ' Fwd IAT Mean', ' Fwd IAT Min', ' Bwd IAT Std', ' Bwd IAT Max', 'Fwd PSH Flags', ' Fwd Header Length', ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean', ' Packet Length Std', ' Packet Length Variance', ' ACK Flag Count', ' URG Flag Count', ' CWE Flag Count', ' Average Packet Size', ' Avg Fwd Segment Size', ' Avg Bwd Segment Size', ' Fwd Header Length.1', 'Subflow Fwd Packets', ' Subflow Bwd Packets', ' Subflow Bwd Bytes', 'Init_Win_bytes_forward', ' Init_Win_bytes_backward', 'Active Mean', ' Active Std', ' Active Max', ' Active Min', 'Idle Mean', ' Idle Min', ' Inbound'],
        },
        "PCA": 0.999,
    },
    "Multiclass": {
        "All": multiclass_X.columns,
        "Correlation": [' Protocol', 'Total Length of Fwd Packets', ' Fwd Packet Length Max', ' Fwd Packet Length Min', ' Fwd Packet Length Mean', ' Bwd Packet Length Min', 'Flow Bytes/s', ' Flow Packets/s', 'Fwd Packets/s', ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean', ' URG Flag Count', ' Down/Up Ratio', ' Average Packet Size', ' Avg Fwd Segment Size', ' Subflow Fwd Bytes', ' Inbound', ' Total Fwd Packets', 'Subflow Fwd Packets', ' act_data_pkt_fwd', ' Flow IAT Mean', ' Flow IAT Std', ' Fwd IAT Mean', ' Fwd IAT Std', ' min_seg_size_forward', ' Flow Duration', ' Flow IAT Max', 'Fwd IAT Total', ' Fwd IAT Max', ' Bwd IAT Mean', ' Bwd IAT Std', ' ACK Flag Count', 'Idle Mean', ' Idle Std', ' Idle Max', ' Fwd Header Length', ' Fwd Header Length.1'],
        "Mutual Information": [' Protocol', ' Flow Duration', ' Total Fwd Packets', ' Total Backward Packets', 'Total Length of Fwd Packets', ' Total Length of Bwd Packets', ' Fwd Packet Length Max', ' Fwd Packet Length Min', ' Fwd Packet Length Mean', ' Fwd Packet Length Std', 'Bwd Packet Length Max', ' Bwd Packet Length Min', ' Bwd Packet Length Mean', 'Flow Bytes/s', ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max', ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std', ' Bwd IAT Max', ' Bwd IAT Min', ' Fwd Header Length', ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s', ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean', ' Packet Length Std', ' Packet Length Variance', ' ACK Flag Count', ' Average Packet Size', ' Avg Fwd Segment Size', ' Avg Bwd Segment Size', ' Fwd Header Length.1', 'Subflow Fwd Packets', ' Subflow Fwd Bytes', ' Subflow Bwd Packets', ' Subflow Bwd Bytes', 'Init_Win_bytes_forward', ' Init_Win_bytes_backward', ' act_data_pkt_fwd', ' min_seg_size_forward', 'Active Mean', ' Active Std', ' Active Max', ' Active Min', 'Idle Mean', ' Idle Std', ' Idle Max', ' Idle Min'],
        "RFE Sets": {
                "Decision Tree": [' Flow Duration', ' Total Fwd Packets', ' Total Backward Packets', 'Total Length of Fwd Packets', ' Total Length of Bwd Packets', ' Fwd Packet Length Max', ' Fwd Packet Length Min', ' Fwd Packet Length Mean', ' Fwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max', ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std', ' Bwd IAT Max', ' Bwd IAT Min', ' Fwd Header Length', ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s', ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean', ' Packet Length Std', ' Packet Length Variance', ' ACK Flag Count', ' Average Packet Size', ' Avg Fwd Segment Size', ' Fwd Header Length.1', 'Subflow Fwd Packets', ' Subflow Fwd Bytes', 'Init_Win_bytes_forward', ' Init_Win_bytes_backward', ' act_data_pkt_fwd', ' min_seg_size_forward', ' Active Max', ' Active Min', 'Idle Mean', ' Idle Max', ' Idle Min', ' Inbound'],
                "Random Forest": [' Flow Duration', ' Total Fwd Packets', 'Total Length of Fwd Packets', ' Fwd Packet Length Max', ' Fwd Packet Length Min', ' Fwd Packet Length Mean', 'Flow Bytes/s', ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max', ' Fwd IAT Min', ' Fwd Header Length', 'Fwd Packets/s', ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean', ' Average Packet Size', ' Avg Fwd Segment Size', ' Fwd Header Length.1', 'Subflow Fwd Packets', ' Subflow Fwd Bytes', 'Init_Win_bytes_forward', ' act_data_pkt_fwd', ' min_seg_size_forward'],
                "Logistic Regression": [' Protocol', ' Flow Duration', 'Total Length of Fwd Packets', ' Fwd Packet Length Min', ' Flow IAT Mean', ' Flow IAT Std', ' Fwd IAT Std', ' Fwd IAT Max', ' Min Packet Length', ' Packet Length Mean', ' Packet Length Std', ' Packet Length Variance', ' Average Packet Size', ' Subflow Fwd Bytes', ' Subflow Bwd Packets', ' act_data_pkt_fwd', 'Idle Mean', ' Idle Max', ' Idle Min']
        },
        "PCA": 0.999,
    }
}

Dump results for use later.

In [6]:
# Save dictionaries
pickle.dump(feature_lists, open("feature_sets.pickle", "wb"))
pickle.dump(search_spaces, open("hyperparameter_search_spaces.pickle", "wb"))